In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM

# Any results you write to the current directory are saved as output.

### Importing Dataset

In [ ]:
def new_dataset(dataset, step_size):
	data_X, data_Y = [], []
	for i in range(len(dataset)-step_size-1): 
		a = dataset[i:(i+step_size), 0]
		data_X.append(a)
		data_Y.append(dataset[i + step_size, 0])
	return np.array(data_X), np.array(data_Y)

In [ ]:
np.random.seed(7)

### Calculate OHLC Average

In [ ]:
dataset = pd.read_csv('../input/Training_data.csv', usecols=[2,3,4,5,6])
#dataset = dataset[dataset['volume_sell']!=0]

OHLC_avg = dataset[['opening_value','highest_value', 'lowest_value', 'settle_value']].mean(axis = 1)
HLC_avg = dataset[['highest_value', 'lowest_value', 'settle_value']].mean(axis = 1)
close_val = dataset[['settle_value']]

### Reshaping data to make it compatible with the input of neural 

In [ ]:
obs = np.arange(1, len(dataset) + 1, 1)

In [ ]:
OHLC_avg = np.reshape(OHLC_avg.values, (len(OHLC_avg),1)) # 1664
scaler = MinMaxScaler(feature_range=(0, 1))
OHLC_avg = scaler.fit_transform(OHLC_avg)


In [ ]:
y =  dataset['volume_sell']
y = y.values
y = scaler.fit_transform(y.reshape(-1,1))

In [ ]:
train_OHLC = int(len(OHLC_avg) * 0.75)
test_OHLC = len(OHLC_avg) - train_OHLC
x_train, x_test = OHLC_avg[0:train_OHLC,:], OHLC_avg[train_OHLC:len(OHLC_avg),:]
y_train, y_test = y[0:train_OHLC,:], y[train_OHLC:len(OHLC_avg),:]

In [ ]:
y_train.shape

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))
step_size = 1

Creating Neural Network Layer by Layer

In [ ]:
model = Sequential()
model.add(LSTM(32, input_shape=(1, step_size), return_sequences = True))
model.add(LSTM(16))
model.add(Dense(1))
model.add(Activation('linear'))
model.compile(loss='mean_squared_error', optimizer='adagrad') # Try SGD, adam, adagrad and compare!!!
model.fit(x_train, y_train,validation_data=(x_test,y_test) , epochs=5, batch_size=1, verbose=2)

In [ ]:
trainPredict = model.predict(x_train)
testPredict = model.predict(x_test)

In [ ]:
trainPredict = scaler.inverse_transform(trainPredict)
y_train = scaler.inverse_transform(y_train)
testPredict = scaler.inverse_transform(testPredict)
y_test = scaler.inverse_transform(y_test)

In [ ]:
# TRAINING RMSE
trainScore = math.sqrt(mean_squared_error(y_train, trainPredict[:,0]))
print('Train RMSE: %.2f' % (trainScore))

# TEST RMSE
testScore = math.sqrt(mean_squared_error(y_test, testPredict[:,0]))
print('Test RMSE: %.2f' % (testScore))

### Generating Output

In [ ]:
df_t = pd.read_csv("../input/Test_data.csv")

In [ ]:
OHLC_avg = df_t[['opening_value', 'highest_value', 'lowest_value', 'settle_value']].mean(axis = 1)
HLC_avg = df_t[['highest_value', 'lowest_value', 'settle_value']].mean(axis = 1)
close_val = df_t[['settle_value']]
OHLC_avg = np.reshape(OHLC_avg.values, (len(OHLC_avg),1)) # 1664
OHLC_avg = scaler.transform(OHLC_avg)
obs = np.arange(1, len(dataset) + 1, 1)
OHLC_avg = scaler.transform(OHLC_avg)
X = np.reshape(OHLC_avg, (OHLC_avg.shape[0], 1, OHLC_avg.shape[1]))
step_size = 1


In [ ]:
preds = model.predict(X)
preds = scaler.inverse_transform(preds)

In [ ]:
preds

In [ ]:
df2 = pd.read_csv("../input/Test_data.csv")

In [ ]:
serial = df2['serial_number']
data = {'serial_number': serial, 'volume_sell': preds[:,0]}
submission = pd.DataFrame(data)
submission.to_csv('Submission.csv', index=False)